In [1]:
from python_speech_features import mfcc

In [2]:
import scipy.io.wavfile as wav

In [3]:
import numpy as np

In [4]:
from tempfile import TemporaryFile

In [5]:
import os

In [6]:
import pickle

In [7]:
import random

In [8]:
import operator

In [9]:
#function to find the distance between feature vectors and find the neighbours
def findNeighbour(trainSet , instance , k):
    #gets the distance between the instance and all the points in the trainset
    dist = []
    for x in range (len(trainSet)):
        tempDist = distance(trainSet[x] , instance , k) + distance(instance , trainSet[x] , k)
        dist.append((trainSet[x][2] , tempDist))
    
    #then we sort distances
    dist.sort(key = operator.itemgetter(1))
    
    #now we get the nearest k neighbours
    neighbours = []
    for x in range(k):
        neighbours.append(dist[x][0])
    return neighbours

In [10]:
#identify the class of the neighbours
def neighbourClass(neighbours):
    classVote = {}
    
    for x in range(len(neighbours)):
        response = neighbours[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
        
    sorter = sorted(classVote.items() , key = operator.itemgetter(1) , reverse = True)
    
    return sorter[0][0]

In [11]:
#to evaluate the model
def accuracy(testSet , prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
            
    return (1.0 * correct) / len(testSet)

In [12]:
directory = "./music_data_set/Data/genres_original/"

In [13]:
f = open("my.dat" , "wb")

i = 0;

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):        
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()

In [14]:
# Split the dataset into training and testing sets respectively
dataset = []

def loadData(filename, split, trainSet, testSet):
    with open('my.dat', 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trainSet.append(dataset[x])
        else:
            testSet.append(dataset[x])

trainingSet = []
testingSet = []
loadData('my.dat', 0.8, trainingSet, testingSet)

In [15]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [16]:
# making predictions using KNN
leng = len(testingSet)
predictions = []
for x in range(leng):
    predictions.append(neighbourClass(findNeighbour(trainingSet, testingSet[x], 5)))

accuracy1 = accuracy(testingSet, predictions)
print(accuracy1*100)

68.68686868686868


In [17]:
testY = []
for x in range(len(testingSet)):
    testY.append(testingSet[x][-1])
testY

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10]

In [18]:
predictions

[1,
 1,
 3,
 1,
 1,
 9,
 1,
 1,
 9,
 1,
 3,
 1,
 4,
 1,
 4,
 1,
 2,
 8,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 6,
 6,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 3,
 3,
 3,
 3,
 3,
 3,
 8,
 1,
 9,
 9,
 4,
 4,
 9,
 4,
 1,
 1,
 2,
 2,
 3,
 9,
 9,
 4,
 8,
 9,
 9,
 6,
 4,
 4,
 9,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 10,
 5,
 5,
 5,
 5,
 5,
 9,
 5,
 10,
 5,
 5,
 6,
 6,
 4,
 6,
 1,
 6,
 1,
 6,
 1,
 1,
 1,
 1,
 6,
 6,
 1,
 6,
 6,
 1,
 3,
 6,
 6,
 6,
 1,
 7,
 7,
 7,
 7,
 7,
 7,
 2,
 7,
 7,
 4,
 4,
 7,
 7,
 7,
 7,
 7,
 7,
 4,
 7,
 7,
 1,
 8,
 4,
 8,
 8,
 8,
 8,
 9,
 8,
 8,
 8,
 8,
 4,
 6,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 3,
 4,
 8,
 9,
 9,
 9,
 4,
 9,
 9,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 6,
 4,
 1,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 5,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 6,
 10,
 10]

In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(testY, predictions)*100)

68.68686868686868


In [20]:
from sklearn.metrics import classification_report
print(classification_report(testY, predictions))

              precision    recall  f1-score   support

           1       0.40      0.62      0.49        16
           2       0.81      0.81      0.81        21
           3       0.75      0.75      0.75        20
           4       0.33      0.29      0.31        21
           5       0.93      0.82      0.87        17
           6       0.67      0.52      0.59        23
           7       1.00      0.80      0.89        20
           8       0.81      0.71      0.76        24
           9       0.54      0.75      0.63        20
          10       0.88      0.88      0.88        16

    accuracy                           0.69       198
   macro avg       0.71      0.69      0.70       198
weighted avg       0.71      0.69      0.69       198



In [21]:
# testing the code with external samples

test_dir = "./music_data_set/Test/"

#hiphop
# test_file = test_dir + "test6.wav"

#blues
# test_file = test_dir + "test4.wav"

#pop
# test_file = test_dir + "test8.wav"

#rock
# test_file = test_dir + "test10.wav"

#jazz
# test_file = test_dir + "test7.wav"

#reggae
# test_file = test_dir + "test9.wav"

#metal
# test_file = test_dir + "test2.wav"

#country
# test_file = test_dir + "test.wav"

#disco
# test_file = test_dir + "test5.wav"

#classical
test_file = test_dir + "test3.wav"

# test_file = test_dir + "unknown11.wav"


In [25]:
from collections import defaultdict
results = defaultdict(int)

results[1] = "HipHop"
results[2] = "Blues"
results[3] = "Pop"
results[4] = "Rock"
results[5] = "Jazz"
results[6] = "Reggae"
results[7] = "Metal"
results[8] = "Country"
results[9] = "Disco"
results[10] = "Classical"

results

defaultdict(int,
            {1: 'HipHop',
             2: 'Blues',
             3: 'Pop',
             4: 'Rock',
             5: 'Jazz',
             6: 'Reggae',
             7: 'Metal',
             8: 'Country',
             9: 'Disco',
             10: 'Classical'})

In [26]:
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)

In [27]:
pred = neighbourClass(findNeighbour(dataset, feature, 4))
print(results[pred])

Classical


In [29]:
test_dir = "./music_data_set/Test/"

for file in os.listdir(test_dir):
    test_file = test_dir + file
    (rate, sig) = wav.read(test_file)
    mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
    covariance = np.cov(np.matrix.transpose(mfcc_feat))
    mean_matrix = mfcc_feat.mean(0)
    feature = (mean_matrix, covariance, i)
    pred = neighbourClass(findNeighbour(dataset, feature, 4))
    print(file  , "\t: " , results[pred])
    

<ipython-input-29-6f1340916bf5>:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  (rate, sig) = wav.read(test_file)


unknown8.wav 	:  Pop
unknown1.wav 	:  Blues
unknown2.wav 	:  Blues
test.wav 	:  Country
unknown9.wav 	:  Pop
unknown10.wav 	:  Pop
test10.wav 	:  Reggae
unknown7.wav 	:  Pop
unknown5.wav 	:  Pop
unknown6.wav 	:  Pop
test9.wav 	:  Reggae
test4.wav 	:  Blues
test6.wav 	:  HipHop
test8.wav 	:  Pop
test3.wav 	:  Classical
unknown3.wav 	:  Pop
unknown4.wav 	:  Pop
test5.wav 	:  Disco
test7.wav 	:  Jazz
test2.wav 	:  Metal
unknown11.wav 	:  Pop
